In [1]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
data = spark.read.csv('/FileStore/tables/Crimes___2001_to_present-9be1b.csv', inferSchema=True, header=True)

In [2]:
data.columns

Out[3]: ['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [3]:
drop_list = ['Dates', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']

data = data.select([column for column in data.columns if column not in drop_list])

In [4]:
data.show(5)

+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
 ID|Case Number| Date| Block|IUCR| Primary Type| Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year| Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
11034701| JA366925|01/01/2001 11:00:...| 016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...| RESIDENCE| false| false| 412| 4| 8| 45| 11| null| null|2001|08/05/2017 03:50:...| null| null| null|
11227287| JB147188|10/08/2017 03:00:...| 092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| RESIDENCE| false| false|2222| 22| 21| 73| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|
11227583| JB147595|03/28/2017 02:00:...| 026XX W 79TH ST|0620| BURGLARY| UNLAWFUL ENTRY| OTHER| false| false| 835| 8| 18| 70| 05| null| null|2017|02/11/2018 03:57:...| null| null| null|
11227293| JB147230|09/09/2017 08:17:...|060XX S EBERHART AVE|0810| THEFT| OVER $500| RESIDENCE| false| false| 313| 3| 20| 42| 06| null| null|2017|02/11/2018 03:57:...| null| null| null|
11227634| JB147599|08/26/2017 10:00:...| 001XX W RANDOLPH ST|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| HOTEL/MOTEL| false| false| 122| 1| 42| 32| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
only showing top 5 rows

In [5]:
data.printSchema()

root
-- ID: integer (nullable = true)
-- Case Number: string (nullable = true)
-- Date: string (nullable = true)
-- Block: string (nullable = true)
-- IUCR: string (nullable = true)
-- Primary Type: string (nullable = true)
-- Description: string (nullable = true)
-- Location Description: string (nullable = true)
-- Arrest: boolean (nullable = true)
-- Domestic: boolean (nullable = true)
-- Beat: integer (nullable = true)
-- District: integer (nullable = true)
-- Ward: integer (nullable = true)
-- Community Area: integer (nullable = true)
-- FBI Code: string (nullable = true)
-- X Coordinate: integer (nullable = true)
-- Y Coordinate: integer (nullable = true)
-- Year: integer (nullable = true)
-- Updated On: string (nullable = true)
-- Latitude: double (nullable = true)
-- Longitude: double (nullable = true)
-- Location: string (nullable = true)

In [6]:
from pyspark.sql.functions import col

# by top 20 categories
data.groupBy("Primary Type") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-------+
 Primary Type| count|
+--------------------+-------+
 THEFT|1494719|
 BATTERY|1291953|
 CRIMINAL DAMAGE| 803703|
 NARCOTICS| 728789|
 ASSAULT| 443370|
 OTHER OFFENSE| 439195|
 BURGLARY| 400016|
 MOTOR VEHICLE THEFT| 325331|
 DECEPTIVE PRACTICE| 287123|
 ROBBERY| 265757|
 CRIMINAL TRESPASS| 201692|
 WEAPONS VIOLATION| 78274|
 PROSTITUTION| 69136|
PUBLIC PEACE VIOL...| 49620|
OFFENSE INVOLVING...| 48486|
 CRIM SEXUAL ASSAULT| 29336|
 SEX OFFENSE| 26785|
INTERFERENCE WITH...| 16983|
 GAMBLING| 14571|
LIQUOR LAW VIOLATION| 14331|
+--------------------+-------+
only showing top 20 rows

In [7]:
crime_type_groups = data.groupBy('Description').count()

In [8]:
crime_type_counts = crime_type_groups.orderBy('count', ascending=False)
counts_pddf = pd.DataFrame(crime_type_counts.rdd.map(lambda l: l.asDict()).collect())

In [9]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_color_codes("pastel")
plt.rcParams["figure.figsize"] = [20, 8]

sns.set(style="whitegrid")
sns.set_color_codes("dark")

#sns.despine(left=True, bottom=True)
type_graph = sns.barplot(x='count', y='Primary Type', data=counts_pddf)
type_graph.set(ylabel="Primary Type", xlabel="Crimes Record Count")
display()

In [10]:
# by top 20 descriptions
data.groupBy("Description") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+------+
 Description| count|
+--------------------+------+
 SIMPLE|832766|
 $500 AND UNDER|575368|
DOMESTIC BATTERY ...|546020|
 TO VEHICLE|390314|
 TO PROPERTY|370541|
 OVER $500|366320|
POSS: CANNABIS 30...|278120|
 FORCIBLE ENTRY|269963|
 AUTOMOBILE|256006|
 FROM BUILDING|239825|
 RETAIL THEFT|180534|
 TELEPHONE THREAT|138915|
 TO LAND|124772|
 POSS: CRACK|120996|
 UNLAWFUL ENTRY|107159|
HARASSMENT BY TEL...|103559|
AGGRAVATED: OTHER...| 96717|
 POSS: HEROIN(WHITE)| 95531|
 ARMED: HANDGUN| 93951|
STRONGARM - NO WE...| 92138|
+--------------------+------+
only showing top 20 rows

In [11]:
crime_type_groups = data.groupBy('Description').count()
crime_type_counts = crime_type_groups.orderBy('count', ascending=False)
counts_pddf = pd.DataFrame(crime_type_counts.rdd.map(lambda l: l.asDict()).collect())

In [12]:
# by top 20 descriptions
data.groupBy("Location Description") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-------+
Location Description| count|
+--------------------+-------+
 STREET|1839454|
 RESIDENCE|1196676|
 APARTMENT| 740756|
 SIDEWALK| 689585|
 OTHER| 269815|
PARKING LOT/GARAG...| 202812|
 ALLEY| 156957|
SCHOOL, PUBLIC, B...| 146198|
 RESIDENCE-GARAGE| 135526|
 SMALL RETAIL STORE| 127841|
RESIDENCE PORCH/H...| 124051|
 RESTAURANT| 114157|
VEHICLE NON-COMME...| 114053|
 GROCERY FOOD STORE| 91466|
 DEPARTMENT STORE| 89679|
 GAS STATION| 76485|
RESIDENTIAL YARD ...| 75064|
CHA PARKING LOT/G...| 56104|
 PARK PROPERTY| 54207|
COMMERCIAL / BUSI...| 51217|
+--------------------+-------+
only showing top 20 rows

In [13]:

crime_type_groups = data.groupBy('Location Description').count()
crime_type_counts = crime_type_groups.orderBy('count', ascending=False)
counts_pddf = pd.DataFrame(crime_type_counts.rdd.map(lambda l: l.asDict()).collect())

In [14]:
display(counts_pddf)

Location Description,count
STREET,1839454
RESIDENCE,1196676
APARTMENT,740756
SIDEWALK,689585
OTHER,269815
PARKING LOT/GARAGE(NON.RESID.),202812
ALLEY,156957
"SCHOOL, PUBLIC, BUILDING",146198
RESIDENCE-GARAGE,135526
SMALL RETAIL STORE,127841


In [15]:
import datetime
from pyspark.sql.functions import *

In [16]:
data = data.withColumn('date_time', to_timestamp('date', 'MM/dd/yyyy hh:mm:ss a'))\
       .withColumn('month', trunc('date_time', 'YYYY')) #adding a month column to be able to view stats on a monthly basis

In [17]:

data.select(['date','date_time', 'month'])\
  .show(n=2, truncate=False)

+----------------------+-------------------+----------+
date |date_time |month |
+----------------------+-------------------+----------+
01/01/2001 11:00:00 AM|2001-01-01 11:00:00|2001-01-01|
10/08/2017 03:00:00 AM|2017-10-08 03:00:00|2017-01-01|
+----------------------+-------------------+----------+
only showing top 2 rows

In [18]:
import datetime


In [19]:
# Extract the "hour" field from the date into a separate column called "hour"
df_hour = data.withColumn('hour', hour(data['date_time']))

In [20]:

# Derive a data frame with crime counts per hour of the day:
hourly_count = data_hour.groupBy(['Primary Type', 'hour']).count().cache()
hourly_total_count = hourly_count.groupBy('hour').sum('count')

In [21]:

hourly_count_pddf = pd.DataFrame(hourly_total_count.select(hourly_total_count['hour'], hourly_total_count['sum(count)'].alias('count'))\
                                .rdd.map(lambda l: l.asDict())\
                                 .collect())

In [22]:
hourly_count_pddf = hourly_count_pddf.sort_values(by='hour')

In [23]:
fig, ax = plt.subplots()
ax.plot(hourly_count_pddf['hour'], hourly_count_pddf['count'], label='Hourly Count')

ax.set(xlabel='Hour of Day', ylabel='Total records',
       title='Overall hourly crime numbers')
ax.grid(b=True, which='both', axis='y')
ax.legend()
display()

In [24]:
hourly_count_pddf = hourly_count_pddf.sort_values(by='hour')
fig, ax = plt.subplots()
ax.plot(hourly_count_pddf['hour'], hourly_count_pddf['count'], label='Hourly Count')

ax.set(xlabel='Hour of Day', ylabel='Total records',
       title='Overall hourly crime numbers')
ax.grid(b=True, which='both', axis='y')
ax.legend()
display()

In [25]:
df_dates = df_hour.withColumn('week_day', dayofweek(df_hour['date_time']))\
                 .withColumn('year_month', month(df_hour['date_time']))\
                 .withColumn('month_day', dayofmonth(df_hour['date_time']))\
                 .withColumn('date_number', datediff(data['date_time'], to_date(lit('2001-01-01'), format='yyyy-MM-dd')))\
                 .cache()

In [26]:

week_day_crime_counts = df_dates.groupBy('week_day').count()

In [27]:

week_day_crime_counts_pddf = pd.DataFrame(week_day_crime_counts.orderBy('week_day').rdd.map(lambda e: e.asDict()).collect())

In [28]:
sns.barplot(data=week_day_crime_counts_pddf, x='week_day', y='count')
display()

In [29]:

year_month_crime_counts = df_dates.groupBy('year_month').count()

In [30]:
year_month_crime_counts_pddf = pd.DataFrame(year_month_crime_counts.orderBy('year_month').rdd.map(lambda e: e.asDict()).collect())


In [31]:
sns.barplot(data=year_month_crime_counts_pddf, y='count', x='year_month')
display()

In [32]:
  from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="Primary Type", outputCol="words", pattern="\\W")

# stop words
add_stopwords = ["http","https","amp","rt","t","c","the"] # standard stop words

stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)

# bag of words count
countVectors = CountVectorizer(inputCol="filtered", outputCol="features", vocabSize=10000, minDF=5)

In [33]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Primary Type", outputCol = "label")

In [34]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, stopwordsRemover, countVectors, label_stringIdx])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(data)
dataset = pipelineFit.transform(data)

In [35]:
dataset.show(5)

+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------------------+--------------------+--------------------+-----+
 ID|Case Number| Date| Block|IUCR| Primary Type| Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year| Updated On|Latitude|Longitude|Location| words| filtered| features|label|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------------------+--------------------+--------------------+-----+
11034701| JA366925|01/01/2001 11:00:...| 016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...| RESIDENCE| false| false| 412| 4| 8| 45| 11| null| null|2001|08/05/2017 03:50:...| null| null| null|[deceptive, pract...|[deceptive, pract...|(48,[11,12],[1.0,...| 8.0|
11227287| JB147188|10/08/2017 03:00:...| 092XX S RACINE AVE|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| RESIDENCE| false| false|2222| 22| 21| 73| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|[crim, sexual, as...|[crim, sexual, as...|(48,[6,22,23],[1....| 15.0|
11227583| JB147595|03/28/2017 02:00:...| 026XX W 79TH ST|0620| BURGLARY| UNLAWFUL ENTRY| OTHER| false| false| 835| 8| 18| 70| 05| null| null|2017|02/11/2018 03:57:...| null| null| null| [burglary]| [burglary]| (48,[8],[1.0])| 6.0|
11227293| JB147230|09/09/2017 08:17:...|060XX S EBERHART AVE|0810| THEFT| OVER $500| RESIDENCE| false| false| 313| 3| 20| 42| 06| null| null|2017|02/11/2018 03:57:...| null| null| null| [theft]| [theft]| (48,[0],[1.0])| 0.0|
11227634| JB147599|08/26/2017 10:00:...| 001XX W RANDOLPH ST|0281|CRIM SEXUAL ASSAULT| NON-AGGRAVATED| HOTEL/MOTEL| false| false| 122| 1| 42| 32| 02| null| null|2017|02/11/2018 03:57:...| null| null| null|[crim, sexual, as...|[crim, sexual, as...|(48,[6,22,23],[1....| 15.0|
+--------+-----------+--------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [36]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 4948198
Test Dataset Count: 2119138

In [37]:
############Logistic Regression using Count Vector Features
# Build the model
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [38]:
predictions = lrModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("Description","Primary Type","Location Description","label","prediction","probability") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+--------------+------------+------------------------------+-----+----------+
 Description|Primary Type| Location Description|label|prediction|
+--------------+------------+------------------------------+-----+----------+
 OVER $500| THEFT| STREET| 0.0| 0.0|
 OVER $500| THEFT| SIDEWALK| 0.0| 0.0|
$500 AND UNDER| THEFT| STREET| 0.0| 0.0|
$500 AND UNDER| THEFT| GROCERY FOOD STORE| 0.0| 0.0|
$500 AND UNDER| THEFT| RESIDENTIAL YARD (FRONT/BACK)| 0.0| 0.0|
$500 AND UNDER| THEFT|CHURCH/SYNAGOGUE/PLACE OF W...| 0.0| 0.0|
 RETAIL THEFT| THEFT| SMALL RETAIL STORE| 0.0| 0.0|
 OVER $500| THEFT| STREET| 0.0| 0.0|
 RETAIL THEFT| THEFT| GROCERY FOOD STORE| 0.0| 0.0|
 RETAIL THEFT| THEFT| DRUG STORE| 0.0| 0.0|
+--------------+------------+------------------------------+-----+----------+
only showing top 10 rows

In [39]:
############Accuracy of Logistic Regression using Count Vector Features
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[21]: 0.983533988041183

In [40]:
#####Model trained using Naive Bayes Classifiers#####
from pyspark.ml.classification import NaiveBayes

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1)

# train the model
model = nb.fit(trainingData)

In [41]:

predictions = model.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Description","Primary Type","Location Description","label","prediction","probability") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+--------------+------------+------------------------------+-----+----------+------------------------------+
 Description|Primary Type| Location Description|label|prediction| probability|
+--------------+------------+------------------------------+-----+----------+------------------------------+
 OVER $500| THEFT| STREET| 0.0| 0.0|[0.932232331479201,8.913894...|
$500 AND UNDER| THEFT| STREET| 0.0| 0.0|[0.932232331479201,8.913894...|
$500 AND UNDER| THEFT| GROCERY FOOD STORE| 0.0| 0.0|[0.932232331479201,8.913894...|
$500 AND UNDER| THEFT| RESIDENTIAL YARD (FRONT/BACK)| 0.0| 0.0|[0.932232331479201,8.913894...|
$500 AND UNDER| THEFT|CHURCH/SYNAGOGUE/PLACE OF W...| 0.0| 0.0|[0.932232331479201,8.913894...|
 RETAIL THEFT| THEFT| SMALL RETAIL STORE| 0.0| 0.0|[0.932232331479201,8.913894...|
 OVER $500| THEFT| STREET| 0.0| 0.0|[0.932232331479201,8.913894...|
 RETAIL THEFT| THEFT| GROCERY FOOD STORE| 0.0| 0.0|[0.932232331479201,8.913894...|
 RETAIL THEFT| THEFT| DRUG STORE| 0.0| 0.0|[0.932232331479201,8.913894...|
 OVER $500| THEFT| RESIDENCE-GARAGE| 0.0| 0.0|[0.932232331479201,8.913894...|
+--------------+------------+------------------------------+-----+----------+------------------------------+
only showing top 10 rows

In [42]:
#####Naive Bayes Accuracy Prediction###
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[25]: 0.999993801661272

In [43]:
#######Accuracy calculation with Random Forest#####
from pyspark.ml.classification import RandomForestClassifier

# Create an initial RandomForest model.
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)

# Train model with Training Data
rfModel = rf.fit(trainingData)

In [44]:
predictions = rfModel.transform(testData)

predictions.filter(predictions['prediction'] == 0) \
    .select("Description","Primary Type","Location Description","label","prediction","probability") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+--------------+------------+------------------------------+-----+----------+------------------------------+
 Description|Primary Type| Location Description|label|prediction| probability|
+--------------+------------+------------------------------+-----+----------+------------------------------+
 OVER $500| THEFT| RESIDENCE-GARAGE| 0.0| 0.0|[0.5667001684411851,0.08160...|
 OVER $500| THEFT|PARKING LOT/GARAGE(NON.RESID.)| 0.0| 0.0|[0.5667001684411851,0.08160...|
 OVER $500| THEFT| SIDEWALK| 0.0| 0.0|[0.5667001684411851,0.08160...|
$500 AND UNDER| THEFT| STREET| 0.0| 0.0|[0.5667001684411851,0.08160...|
 OVER $500| THEFT| STREET| 0.0| 0.0|[0.5667001684411851,0.08160...|
$500 AND UNDER| THEFT| RESIDENTIAL YARD (FRONT/BACK)| 0.0| 0.0|[0.5667001684411851,0.08160...|
 ATTEMPT THEFT| THEFT| SMALL RETAIL STORE| 0.0| 0.0|[0.5667001684411851,0.08160...|
 RETAIL THEFT| THEFT| SMALL RETAIL STORE| 0.0| 0.0|[0.5667001684411851,0.08160...|
 OVER $500| THEFT| STREET| 0.0| 0.0|[0.5667001684411851,0.08160...|
 OVER $500| THEFT| STREET| 0.0| 0.0|[0.5667001684411851,0.08160...|
+--------------+------------+------------------------------+-----+----------+------------------------------+
only showing top 10 rows

In [45]:
######### Result of Random Forest#####
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

Out[30]: 0.7710231193725561